## Load sub

The videos are in the order of Experiment_id, so not in the order of presentation. This means the first video is the same for each participant.

In [1]:
import pickle
import numpy as np
import gc  # Garbage collector

!pip install mrmr-selection
!pip install ordpy

In [2]:
participants = 32
subjects = {'data': [], 'labels': []}
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'

# Read and collect data
for i in range(1, participants + 1):
    file_name = prefix_path + f"data_preprocessed_python\\s{'0' if i < 10 else ''}{i}.dat"
    with open(file_name, 'rb') as file:
        subject = pickle.load(file, encoding='latin1')
        for key in subjects:
            subjects[key].append(subject[key])

        del subject

# Merge data and reshape
for key in subjects:
    subjects[key] = np.concatenate(subjects[key], axis=0)

print(subjects['data'].shape)
print(subjects['labels'].shape)

(1280, 40, 8064)
(1280, 4)


In [3]:
# Use 2 classifiers valence and arousal
subjects['labels'] = subjects['labels'][:, :2]

# Use threshold 5 to assign binary label to each classifier
subjects['labels'][:, 0] = (subjects['labels'][:, 0] >= 5).astype(int)
subjects['labels'][:, 1] = (subjects['labels'][:, 1] >= 5).astype(int)

print(set(subjects['labels'][:, 0]))
print(set(subjects['labels'][:, 1]))

{np.float64(0.0), np.float64(1.0)}
{np.float64(0.0), np.float64(1.0)}


## Split Train Test Dataset

In [4]:
from sklearn.model_selection import train_test_split

test_size = 0.2
random_state = 42

X_train, X_test, y_train, y_test = train_test_split(
    subjects['data'], subjects['labels'], test_size=test_size, random_state=random_state
)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

del subjects
gc.collect()  # Free up memory

(1024, 40, 8064) (256, 40, 8064) (1024, 2) (256, 2)


0

## Feature extraction


### Wavelet transform

In [5]:
# Wavelet transform
import pywt
from tqdm import tqdm

# Sampling frequency:
fs = 128

# Select scales and wavelets
scales = np.arange(1, 129)  # Example: 129 scale values
wavelet = 'morl'

def CWT_for_sample(data):
    cwt_output = []
    # Calculate CWT for each sample and each channel
    for i in tqdm(range(data.shape[0])):  # 1024 samples
        cwt_channels = []
        for j in range(data.shape[1]):  # 40 channels
            coeffs, freqs = pywt.cwt(data[i, j], scales, wavelet) # coeffs are of the form (len(scales), 8064)
            energy_scales = np.sum(np.abs(coeffs)**2, axis=1)  # (len(scales),)
            cwt_channels.append(energy_scales)
        cwt_output.append(cwt_channels)
    
    # Convert output to numpy array
    cwt_output = np.array(cwt_output)  # Size: (1024, 40, len(scales))

    return cwt_output
    
# cwt_train = CWT_for_sample(X_train) # Size: (1024, 40, len(scales))
# cwt_test = CWT_for_sample(X_test) # Size: (256, 40, len(scales))

# print(cwt_train.shape, cwt_test.shape)

# # Save file name
# prefix_output_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
# filename = "cwt_data.pkl"

# # Save data
# with open(prefix_output_path +filename, 'wb') as file:
#     pickle.dump({'train': cwt_train, 'test': cwt_test}, file)

# print(f"Data has been saved to file {filename}")


In [6]:
# Save file name
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
filename = "cwt_data.pkl"

# Read data again
with open(prefix_path + filename, 'rb') as file:
    data = pickle.load(file)

# Data retrieval
cwt_train_loaded = data['train']
cwt_test_loaded = data['test']

print("Size cwt_train:", cwt_train_loaded.shape)
print("Size cwt_test:", cwt_test_loaded.shape)


Size cwt_train: (1024, 40, 128)
Size cwt_test: (256, 40, 128)


### Nonlinear Feature Analyses: Permutation Entropy

In [7]:
import ordpy

def split_array(arr, num_epochs):
    
    len_arr = len(arr)
    len_epochs = round(len_arr / num_epochs)
    
    splits_arr = [arr[i * len_epochs: len_arr if i + 1 == num_epochs else (i + 1) * len_epochs] for i in range(num_epochs)]

    return splits_arr

def permutation_entropy_eeg(data, num_epochs=8):
    B, C, T = data.shape
    pe = np.zeros((B, C, num_epochs))
    
    for b in tqdm(range(B)):
        for c in range(C):
            splits_arr = split_array(data[b, c], num_epochs)
            for i in range(num_epochs):
                pe[b, c, i] = ordpy.permutation_entropy(splits_arr[i])

    return pe

# pe_train = permutation_entropy_eeg(X_train)#1024x40
# pe_test = permutation_entropy_eeg(X_test)#256x40

# # Save file name
# prefix_output_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
# filename = "pe_data.pkl"

# # Save data
# with open(prefix_output_path + filename, 'wb') as file:
#    pickle.dump({'train': pe_train, 'test': pe_test}, file)

# print(f"Data has been saved to file {filename}")

In [8]:
# # Save file name
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
filename = "pe_data.pkl"

# Read data again
with open(prefix_path + filename, 'rb') as file:
    data = pickle.load(file)

# Data retrieval
pe_train_loaded = data['train']
pe_test_loaded = data['test']

print("Size pe_train:", pe_train_loaded.shape)
print("Size pe_test:", pe_test_loaded.shape)

Size pe_train: (1024, 40, 8)
Size pe_test: (256, 40, 8)


## Feature Selection

In [9]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import pickle

DEVICE = 'cpu' 

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, rnn_type="LSTM", device=DEVICE):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device

        # Choose the RNN type (LSTM or GRU)
        if rnn_type == "LSTM":
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        elif rnn_type == "GRU":
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        else:
            raise ValueError("Unsupported RNN type. Use 'LSTM' or 'GRU'.")

        # Fully connected output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, feature_extraction=False):
        batch_size = x.size(0)
        seq_len = x.size(1)

        # Reshape if necessary
        if len(x.shape) == 2:
            x = x.view(batch_size, seq_len, self.input_size)

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device) if isinstance(self.rnn, nn.LSTM) else None

        # Pass through the RNN
        out, (hn, cn) = self.rnn(x, (h0, c0)) if isinstance(self.rnn, nn.LSTM) else self.rnn(x, h0)

        if feature_extraction:
            return out

        # Take the last hidden state
        out = out[:, -1, :]

        # Output layer
        out = self.fc(out)
        return out


class CNNModel(nn.Module):
    def __init__(self, in_channels, outs_channels, output_size, length_signal, device=DEVICE):
        super().__init__()

        channels = [in_channels] + outs_channels
        self.model = nn.Sequential(
            *[nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, device=device),
                nn.BatchNorm1d(out_channels, device=device),
                nn.ReLU()
            ) for in_channels, out_channels in zip(channels[:-1], channels[1:]) ]
        )

        self.fc = nn.Linear(outs_channels[-1] * length_signal, output_size, device=device) 

    def forward(self, x, feature_extraction=False):
        x = self.model(x)

        if feature_extraction:
            # If we want feature extraction, return features before the fully connected layer
            return x

        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = x.squeeze()

        return x


def pca_data(data, k):
    # Reshape the data into (n_samples, n_features) to prepare for PCA
    n_samples, n_channels, n_features = data.shape
    data_reshaped = data.reshape(n_samples * n_channels, n_features)
    
    # Data Normalization
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_reshaped)
    
    # Apply PCA to reduce dimensionality
    pca = PCA(n_components=k)
    data_pca = pca.fit_transform(data_scaled)
    
    # Reshape back to (n_samples, n_channels, k)
    data_pca_reshaped = data_pca.reshape(n_samples, n_channels, k)
    
    return data_pca_reshaped


def feature_extraction_with_rnn_cnn(X, y, model_type="CNN", k=128):
    """
    Extract features from the RNN or CNN model and return the extracted features.
    
    Args:
        X (numpy.ndarray): The input data (n_samples, n_channels, n_features).
        y (numpy.ndarray): The labels (n_samples, num_classes).
        model_type (str): The model to use for feature extraction, either "RNN" or "CNN".
        k (int): Number of principal components to keep after PCA.

    Returns:
        extracted_features (numpy.ndarray): The extracted features from the model.
    """
    # Apply PCA
    pca_data_transformed = pca_data(X, k=k)

    # Convert to tensor
    pca_data_tensor = torch.Tensor(pca_data_transformed).to(DEVICE)
    
    # Reshape the data to have 1 channel instead of 40
    pca_data_tensor = pca_data_tensor.reshape(pca_data_tensor.shape[0], 1, pca_data_tensor.shape[1] * pca_data_tensor.shape[2])

    # Choose model
    if model_type == "CNN":
        model = CNNModel(in_channels=1, outs_channels=[32, 64], output_size=10, length_signal=k, device=DEVICE)
    elif model_type == "RNN":
        model = RNNModel(input_size=5120, hidden_size=128, num_layers=2, output_size=10, device=DEVICE)
    else:
        raise ValueError("Model type must be either 'RNN' or 'CNN'.")

    # Extract features using the model
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        extracted_features = model(pca_data_tensor, feature_extraction=True)  # Extract features

    return extracted_features


# Assuming you have X_train, y_train, X_test, and y_test already loaded

# Select model type based on input
model_type = "RNN" 

# Feature extraction for train and test sets
if model_type == "CNN":
    features_train = feature_extraction_with_rnn_cnn(X_train, y_train, model_type="CNN", k=128)
    features_test = feature_extraction_with_rnn_cnn(X_test, y_test, model_type="CNN", k=128)
elif model_type == "RNN":
    features_train = feature_extraction_with_rnn_cnn(X_train, y_train, model_type="RNN", k=128)
    features_test = feature_extraction_with_rnn_cnn(X_test, y_test, model_type="RNN", k=128)

# Save the extracted features to a pickle file
output_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
filename = f"pca_data_train_test_{model_type.lower()}.pkl"  # File name will include model type

with open(output_path + filename, 'wb') as file:
    pickle.dump({
        'train': features_train,
        'test': features_test
    }, file)

print(f"Data has been saved to file {output_path + filename}")


Data has been saved to file C:\Users\ferri\Downloads\PoliTO\Tesi\DSs\Emotion-Stress\deap-dataset\pca_data_train_test_rnn.pkl


In [10]:
# Save file name
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
if model_type == "CNN":
    filename = "pca_data_train_test_cnn.pkl"
elif model_type == "RNN":
    filename = "pca_data_train_test_rnn.pkl"

# Read data again
with open(prefix_path + filename, 'rb') as file:
    data = pickle.load(file)
    
# print(data)

# Data retrieval
pca_train_loaded = data['train']
pca_test_loaded = data['test']

print("Size pca_train:", pca_train_loaded.shape)
print("Size pca_test:", pca_test_loaded.shape)

Size pca_train: torch.Size([1024, 1, 128])
Size pca_test: torch.Size([256, 1, 128])


In [11]:
from mrmr import mrmr_classif
import pandas as pd

def feature_selector(num_features, X, y, classif_obj): # keep last dimension
    flatted_X = X.reshape(-1, X.shape[-1])

    flatted_X = pd.DataFrame(flatted_X)

    flatted_y_new = y
    if len(y.shape) == 2:
        y_new = np.repeat(y[:, np.newaxis, :], X.shape[1], axis=1)
        flatted_y_new = y_new.reshape(-1, y_new.shape[-1])
        
    flatted_y_new = flatted_y_new[:,classif_obj]

    selected_features = mrmr_classif(X=flatted_X, y=flatted_y_new, K=num_features)

    return selected_features

f = feature_selector(64, cwt_train_loaded, y_train, 0)
feature_selections = {
    'PCA': {
        'Valence': feature_selector(64, pca_train_loaded, y_train, 0), #1024x40x8064
        'Arousal': feature_selector(64, pca_train_loaded, y_train, 1) #1024x40x8064
    },
    'WT': {
        'Valence': feature_selector(64, cwt_train_loaded, y_train, 0), #1024x40x128
        'Arousal': feature_selector(64, cwt_train_loaded, y_train, 1) #1024x40x128
    }
}

# Save file name
prefix_output_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
filename = "feature_selections.pkl"

# Save data
with open(prefix_output_path + filename, 'wb') as file:
    pickle.dump(feature_selections, file)

print(f"Data has been saved to file {filename}")

100%|██████████| 64/64 [00:13<00:00,  4.67it/s]

Data has been saved to file feature_selections.pkl


In [12]:
# Save file name
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
filename = "feature_selections.pkl"

# Read data again
with open(prefix_path + filename, 'rb') as file:
    feature_selection_loaded = pickle.load(file)

print(feature_selection_loaded)

{'PCA': {'Valence': [np.int64(117), np.int64(67), np.int64(42), np.int64(4), np.int64(44), np.int64(111), np.int64(81), np.int64(119), np.int64(99), np.int64(14), np.int64(54), np.int64(110), np.int64(45), np.int64(87), np.int64(48), np.int64(38), np.int64(40), np.int64(66), np.int64(98), np.int64(76), np.int64(70), np.int64(94), np.int64(2), np.int64(118), np.int64(28), np.int64(9), np.int64(121), np.int64(24), np.int64(125), np.int64(47), np.int64(23), np.int64(51), np.int64(95), np.int64(61), np.int64(30), np.int64(75), np.int64(101), np.int64(39), np.int64(29), np.int64(52), np.int64(22), np.int64(68), np.int64(57), np.int64(79), np.int64(7), np.int64(90), np.int64(6), np.int64(59), np.int64(127), np.int64(37), np.int64(97), np.int64(25), np.int64(73), np.int64(77), np.int64(85), np.int64(0), np.int64(36), np.int64(93), np.int64(113), np.int64(108), np.int64(62), np.int64(104), np.int64(5), np.int64(17)], 'Arousal': [np.int64(76), np.int64(1), np.int64(15), np.int64(123), np.int64(

## Data synthesis

In [13]:
aggregated_data = [
    ['WT', 'YES', 'VALENCE', {'TRAIN': (cwt_train_loaded[:, :, feature_selection_loaded['WT']['Valence']], y_train), 
                              'TEST': (cwt_test_loaded[:, :, feature_selection_loaded['WT']['Valence']], y_test)}],
    ['WT', 'YES', 'AROUSAL', {'TRAIN': (cwt_train_loaded[:, :, feature_selection_loaded['WT']['Arousal']], y_train),
                              'TEST': (cwt_test_loaded[:, :, feature_selection_loaded['WT']['Arousal']], y_test)}],

    ['WT', 'NO', '*', {'TRAIN': (cwt_train_loaded, y_train), 
                       'TEST': (cwt_test_loaded, y_test)}],

    ['PE', 'NO', '*', {'TRAIN': (pe_train_loaded, y_train), 
                       'TEST': (pe_test_loaded, y_test)}],

    ['NONE', 'YES', 'VALENCE', {'TRAIN': (pca_train_loaded[:, :, feature_selection_loaded['PCA']['Valence']], y_train), 
                                'TEST': (pca_test_loaded[:, :, feature_selection_loaded['PCA']['Valence']], y_test)}],
    ['NONE', 'YES', 'AROUSAL', {'TRAIN': (pca_train_loaded[:, :, feature_selection_loaded['PCA']['Arousal']], y_train), 
                                'TEST': (pca_test_loaded[:, :, feature_selection_loaded['PCA']['Arousal']], y_test)}],

    ['NONE', 'NO', '*', {'TRAIN': (pca_train_loaded, y_train), 
                         'TEST': (pca_test_loaded, y_test)}],
]

## Model

In [15]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from torch.optim.lr_scheduler import StepLR

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def calculate_metric(metric, y_true, y_pred):
    """
    Calculates the value of a metric based on the input metric name.

    Args:
        metric (str): Metric name, one of the ['Acc', 'Sens', 'Spec', 'Prec', 'F-measure'].
        y_true (array-like): Actual label.
        y_pred (array-like): Predictive label.

    Returns:
        float: Value of selected metric.

    Raises:
        ValueError: If the metric is not in the supported list.
    """
    metric = metric.lower()  # Convert metrics to lowercase for easier handling

    if metric == 'acc':
        return accuracy_score(y_true, y_pred)
    elif metric == 'sens':  # Sensitivity (Recall)
        return recall_score(y_true, y_pred, average='binary')
    elif metric == 'spec':  # Specificity
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        return tn / (tn + fp) if (tn + fp) != 0 else 0.0
    elif metric == 'prec':  # Precision
        return precision_score(y_true, y_pred, average='binary')
    elif metric == 'f-measure':  # F1-Score
        return f1_score(y_true, y_pred, average='binary')
    else:
        raise ValueError(f"Metric '{metric}' not supported. Please select one of the ['Acc', 'Sens', 'Spec', 'Prec', 'F-measure'].")
    

def evaluate(model, name_model, classifi_obj, X, y):
    """
    Evaluate the model on the provided data.

    Args:
        model: The trained model.
        name_model: The type of model ('rnn', 'cnn', etc.)
        classifi_obj: Whether it's a classification problem (True/False).
        X: Input data.
        y: True labels.

    Returns:
        dict: Metrics like accuracy, sensitivity, specificity, etc.
    """
    if name_model == 'rnn' or name_model == 'cnn':
        # Handle PyTorch models (RNN, CNN, etc.)
        model.eval()  # Set model to evaluation mode if it's a PyTorch model

        if classifi_obj:
            X = X.permute(0, 2, 1)  # Assuming the model expects input (batch_size, seq_len, feature_dim)
        
        with torch.no_grad():
            y_pred = model(X)  # PyTorch models are callable
            y_pred = F.sigmoid(y_pred) >= 0.5  # For binary classification (change if necessary)
            y_pred = y_pred.float().cpu().numpy()

    else:
        # Handle scikit-learn models (RandomForest, etc.)
        # If the model is not a PyTorch model, reshape the input to 2D (n_samples, n_features)
        n_samples, n_channels, n_features = X.shape
        X_reshaped = X.reshape(n_samples, n_channels * n_features)  # Flatten the channels and features
        
        y_pred = model.predict(X_reshaped)  # Use the `predict` method for scikit-learn models
        y_pred = y_pred.astype(float)  # Ensure the output is a float for metric calculations

    # Convert y_true to a 1D array if it's multilabel-indicator
    if len(y.shape) > 1 and y.shape[1] > 1:  # Check if y_true is multilabel (e.g., one-hot encoded)
        y = np.argmax(y, axis=1)  # Convert to class labels

    # Calculate metrics (accuracy, precision, etc.)
    result = {
        "Acc": calculate_metric("Acc", y, y_pred),
        "Sens": calculate_metric("Sens", y, y_pred),
        "Spec": calculate_metric("Spec", y, y_pred),
        "Prec": calculate_metric("Prec", y, y_pred),
        "F-measure": calculate_metric("F-measure", y, y_pred),
    }

    return result

In [16]:
def setup_config_train_ml_model(X_train, y_train, X_test, y_test, name_model):
    return {
        'X_train': X_train,
        'y_train': y_train,
        'name_model': name_model.lower()
    }

def train_ml_model(model, classifi_obj, **kwargs):
    name_model = kwargs.get('name_model').lower()
    X_train = kwargs.get('X_train')
    X_train = X_train.reshape(X_train.shape[0], -1)
    y = kwargs.get('y_train')[:, classifi_obj]
    model.fit(X_train, y)

def setup_config_train(X_train, y_train, X_test, y_test, name_model):
    name_model = name_model.lower()
    return setup_config_train_ml_model(X_train, y_train, X_test, y_test, name_model)

def train_model(model, classifi_obj, **kwargs):
    if kwargs.get('name_model').lower() in ['svm', 'rf', 'lr', 'knn', 'gb', 'xgboost', 'adaboost', 'dt']:

        return train_ml_model(model, classifi_obj, **kwargs)
    

In [17]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb

def get_model(**kwargs):
    model = None
    name_model = kwargs.get('name_model').lower()
    
    device = kwargs.get('device', DEVICE)
    
    if name_model == 'svm':
        model = SVC(kernel='rbf')
        
    elif name_model == 'rf':
        model = RandomForestClassifier(n_estimators=100, max_depth=20, min_samples_leaf=5, random_state=42)

    elif name_model == 'lr':
        model = LogisticRegression(max_iter=1000, random_state=42)

    elif name_model == 'knn':
        model = KNeighborsClassifier(n_neighbors=5)

    elif name_model == 'dt':
        model = DecisionTreeClassifier(max_depth=10, random_state=42)

    else:
        raise ValueError(f"Model {name_model} not recognized. Please choose from 'svm', 'rf', 'lr', 'knn', 'gb', 'xgboost', 'adaboost', 'dt'.")
    
    return model


In [18]:
models = []
evaluates = []
i = 0
for name_model in ['rf', 'svm', 'lr', 'knn', 'dt']:
    for experiment in aggregated_data:
        data = experiment[-1]
        classifi_objs = [0] if experiment[2] == 'VALENCE' else [1] if experiment[2] == 'AROUSAL' else [0,1]
        (X_train_data, y_train_data), (X_test_data, y_test_data) = data['TRAIN'], data['TEST']
        # print(X_test_data.shape, X_train_data.shape)
        for classifi_obj in classifi_objs:
            config = {
                'length_signal': X_train_data.shape[-1],
                'name_model': name_model
            }
            
            model = get_model(**config)

            # print(name_model)
            config_train = setup_config_train(X_train_data, y_train_data, X_test_data, y_test_data, name_model)
            train_model(model, classifi_obj, **config_train)

            motion = 'VALENCE' if classifi_obj else 'AROUSAL'

            models.append((*experiment[:2], motion, model))

            info = (
                name_model,
                *experiment[:2], 
                motion,
                evaluate(model, name_model, classifi_obj, X_test_data, y_test_data)['Acc']*100
            )
            i += 1
            print(f'experiment {i}: {info}')
            evaluates.append(info)
            
            

experiment 1: ('rf', 'WT', 'YES', 'AROUSAL', 73.828125)
experiment 2: ('rf', 'WT', 'YES', 'VALENCE', 78.90625)
experiment 3: ('rf', 'WT', 'NO', 'AROUSAL', 74.21875)
experiment 4: ('rf', 'WT', 'NO', 'VALENCE', 77.734375)
experiment 5: ('rf', 'PE', 'NO', 'AROUSAL', 78.125)
experiment 6: ('rf', 'PE', 'NO', 'VALENCE', 78.515625)
experiment 7: ('rf', 'NONE', 'YES', 'AROUSAL', 78.515625)
experiment 8: ('rf', 'NONE', 'YES', 'VALENCE', 78.515625)
experiment 9: ('rf', 'NONE', 'NO', 'AROUSAL', 78.515625)
experiment 10: ('rf', 'NONE', 'NO', 'VALENCE', 78.515625)
experiment 11: ('svm', 'WT', 'YES', 'AROUSAL', 77.734375)
experiment 12: ('svm', 'WT', 'YES', 'VALENCE', 78.515625)
experiment 13: ('svm', 'WT', 'NO', 'AROUSAL', 77.734375)
experiment 14: ('svm', 'WT', 'NO', 'VALENCE', 78.515625)
experiment 15: ('svm', 'PE', 'NO', 'AROUSAL', 78.515625)
experiment 16: ('svm', 'PE', 'NO', 'VALENCE', 78.515625)
experiment 17: ('svm', 'NONE', 'YES', 'AROUSAL', 78.515625)
experiment 18: ('svm', 'NONE', 'YES', 

In [19]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import pickle

DEVICE = 'cpu' 

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, rnn_type="GRU", device=DEVICE):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.device = device

        # Choose the RNN type (LSTM or GRU)
        if rnn_type == "LSTM":
            self.rnn = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        elif rnn_type == "GRU":
            self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        else:
            raise ValueError("Unsupported RNN type. Use 'LSTM' or 'GRU'.")

        # Fully connected output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, feature_extraction=False):
        batch_size = x.size(0)
        seq_len = x.size(1)

        # Reshape if necessary
        if len(x.shape) == 2:
            x = x.view(batch_size, seq_len, self.input_size)

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(self.device) if isinstance(self.rnn, nn.LSTM) else None

        # Pass through the RNN
        out, (hn, cn) = self.rnn(x, (h0, c0)) if isinstance(self.rnn, nn.LSTM) else self.rnn(x, h0)

        if feature_extraction:
            return out

        # Take the last hidden state
        out = out[:, -1, :]

        # Output layer
        out = self.fc(out)
        return out


class CNNModel(nn.Module):
    def __init__(self, in_channels, outs_channels, output_size, length_signal, device=DEVICE):
        super().__init__()

        channels = [in_channels] + outs_channels
        self.model = nn.Sequential(
            *[nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, device=device),
                nn.BatchNorm1d(out_channels, device=device),
                nn.ReLU()
            ) for in_channels, out_channels in zip(channels[:-1], channels[1:]) ]
        )

        self.fc = nn.Linear(outs_channels[-1] * length_signal, output_size, device=device) 

    def forward(self, x, feature_extraction=False):
        x = self.model(x)

        if feature_extraction:
            # If we want feature extraction, return features before the fully connected layer
            return x

        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = x.squeeze()

        return x


def pca_data(data, k):
    # Reshape the data into (n_samples, n_features) to prepare for PCA
    n_samples, n_channels, n_features = data.shape
    data_reshaped = data.reshape(n_samples * n_channels, n_features)
    
    # Data Normalization
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_reshaped)
    
    # Apply PCA to reduce dimensionality
    pca = PCA(n_components=k)
    data_pca = pca.fit_transform(data_scaled)
    
    # Reshape back to (n_samples, n_channels, k)
    data_pca_reshaped = data_pca.reshape(n_samples, n_channels, k)
    
    return data_pca_reshaped


def feature_extraction_with_rnn_cnn(X, y, model_type="CNN", k=128):
    """
    Extract features from the RNN or CNN model and return the extracted features.
    
    Args:
        X (numpy.ndarray): The input data (n_samples, n_channels, n_features).
        y (numpy.ndarray): The labels (n_samples, num_classes).
        model_type (str): The model to use for feature extraction, either "RNN" or "CNN".
        k (int): Number of principal components to keep after PCA.

    Returns:
        extracted_features (numpy.ndarray): The extracted features from the model.
    """
    # Apply PCA
    pca_data_transformed = pca_data(X, k=k)

    # Convert to tensor
    pca_data_tensor = torch.Tensor(pca_data_transformed).to(DEVICE)
    
    # Reshape the data to have 1 channel instead of 40
    pca_data_tensor = pca_data_tensor.reshape(pca_data_tensor.shape[0], 1, pca_data_tensor.shape[1] * pca_data_tensor.shape[2])

    # Choose model
    if model_type == "CNN":
        model = CNNModel(in_channels=1, outs_channels=[32, 64], output_size=10, length_signal=k, device=DEVICE)
    elif model_type == "RNN":
        model = RNNModel(input_size=5120, hidden_size=128, num_layers=2, output_size=10, device=DEVICE)
    else:
        raise ValueError("Model type must be either 'RNN' or 'CNN'.")

    # Extract features using the model
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        extracted_features = model(pca_data_tensor, feature_extraction=True)  # Extract features

    return extracted_features


# Assuming you have X_train, y_train, X_test, and y_test already loaded

# Select model type based on input
model_type = "RNN" 

# Feature extraction for train and test sets
if model_type == "CNN":
    features_train = feature_extraction_with_rnn_cnn(X_train, y_train, model_type="CNN", k=128)
    features_test = feature_extraction_with_rnn_cnn(X_test, y_test, model_type="CNN", k=128)
elif model_type == "RNN":
    features_train = feature_extraction_with_rnn_cnn(X_train, y_train, model_type="RNN", k=128)
    features_test = feature_extraction_with_rnn_cnn(X_test, y_test, model_type="RNN", k=128)

# Save the extracted features to a pickle file
output_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
filename = f"pca_data_train_test_{model_type.lower()}.pkl"  # File name will include model type

with open(output_path + filename, 'wb') as file:
    pickle.dump({
        'train': features_train,
        'test': features_test
    }, file)

print(f"Data has been saved to file {output_path + filename}")


Data has been saved to file C:\Users\ferri\Downloads\PoliTO\Tesi\DSs\Emotion-Stress\deap-dataset\pca_data_train_test_rnn.pkl


In [20]:
# Save file name
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
if model_type == "CNN":
    filename = "pca_data_train_test_cnn.pkl"
elif model_type == "RNN":
    filename = "pca_data_train_test_rnn.pkl"

# Read data again
with open(prefix_path + filename, 'rb') as file:
    data = pickle.load(file)
    
# print(data)

# Data retrieval
pca_train_loaded = data['train']
pca_test_loaded = data['test']

print("Size pca_train:", pca_train_loaded.shape)
print("Size pca_test:", pca_test_loaded.shape)

Size pca_train: torch.Size([1024, 1, 128])
Size pca_test: torch.Size([256, 1, 128])


In [21]:
from mrmr import mrmr_classif
import pandas as pd

def feature_selector(num_features, X, y, classif_obj): # keep last dimension
    flatted_X = X.reshape(-1, X.shape[-1])

    flatted_X = pd.DataFrame(flatted_X)

    flatted_y_new = y
    if len(y.shape) == 2:
        y_new = np.repeat(y[:, np.newaxis, :], X.shape[1], axis=1)
        flatted_y_new = y_new.reshape(-1, y_new.shape[-1])
        
    flatted_y_new = flatted_y_new[:,classif_obj]

    selected_features = mrmr_classif(X=flatted_X, y=flatted_y_new, K=num_features)

    return selected_features

f = feature_selector(64, cwt_train_loaded, y_train, 0)
feature_selections = {
    'PCA': {
        'Valence': feature_selector(64, pca_train_loaded, y_train, 0), #1024x40x8064
        'Arousal': feature_selector(64, pca_train_loaded, y_train, 1) #1024x40x8064
    },
    'WT': {
        'Valence': feature_selector(64, cwt_train_loaded, y_train, 0), #1024x40x128
        'Arousal': feature_selector(64, cwt_train_loaded, y_train, 1) #1024x40x128
    }
}

# Save file name
prefix_output_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
filename = "feature_selections.pkl"

# Save data
with open(prefix_output_path + filename, 'wb') as file:
    pickle.dump(feature_selections, file)

print(f"Data has been saved to file {filename}")

100%|██████████| 64/64 [00:13<00:00,  4.64it/s]

Data has been saved to file feature_selections.pkl


In [22]:
# Save file name
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
filename = "feature_selections.pkl"

# Read data again
with open(prefix_path + filename, 'rb') as file:
    feature_selection_loaded = pickle.load(file)

print(feature_selection_loaded)

{'PCA': {'Valence': [np.int64(45), np.int64(124), np.int64(36), np.int64(17), np.int64(56), np.int64(79), np.int64(12), np.int64(32), np.int64(102), np.int64(26), np.int64(63), np.int64(66), np.int64(105), np.int64(58), np.int64(70), np.int64(127), np.int64(117), np.int64(59), np.int64(103), np.int64(89), np.int64(65), np.int64(106), np.int64(107), np.int64(72), np.int64(48), np.int64(86), np.int64(0), np.int64(51), np.int64(74), np.int64(55), np.int64(90), np.int64(119), np.int64(52), np.int64(10), np.int64(88), np.int64(97), np.int64(104), np.int64(29), np.int64(123), np.int64(2), np.int64(116), np.int64(101), np.int64(62), np.int64(64), np.int64(35), np.int64(5), np.int64(85), np.int64(38), np.int64(99), np.int64(31), np.int64(113), np.int64(19), np.int64(118), np.int64(120), np.int64(71), np.int64(11), np.int64(109), np.int64(1), np.int64(95), np.int64(28), np.int64(81), np.int64(100), np.int64(77), np.int64(40)], 'Arousal': [np.int64(32), np.int64(69), np.int64(34), np.int64(109),

In [23]:
aggregated_data = [
    ['WT', 'YES', 'VALENCE', {'TRAIN': (cwt_train_loaded[:, :, feature_selection_loaded['WT']['Valence']], y_train), 
                              'TEST': (cwt_test_loaded[:, :, feature_selection_loaded['WT']['Valence']], y_test)}],
    ['WT', 'YES', 'AROUSAL', {'TRAIN': (cwt_train_loaded[:, :, feature_selection_loaded['WT']['Arousal']], y_train),
                              'TEST': (cwt_test_loaded[:, :, feature_selection_loaded['WT']['Arousal']], y_test)}],

    ['WT', 'NO', '*', {'TRAIN': (cwt_train_loaded, y_train), 
                       'TEST': (cwt_test_loaded, y_test)}],

    ['PE', 'NO', '*', {'TRAIN': (pe_train_loaded, y_train), 
                       'TEST': (pe_test_loaded, y_test)}],

    ['NONE', 'YES', 'VALENCE', {'TRAIN': (pca_train_loaded[:, :, feature_selection_loaded['PCA']['Valence']], y_train), 
                                'TEST': (pca_test_loaded[:, :, feature_selection_loaded['PCA']['Valence']], y_test)}],
    ['NONE', 'YES', 'AROUSAL', {'TRAIN': (pca_train_loaded[:, :, feature_selection_loaded['PCA']['Arousal']], y_train), 
                                'TEST': (pca_test_loaded[:, :, feature_selection_loaded['PCA']['Arousal']], y_test)}],

    ['NONE', 'NO', '*', {'TRAIN': (pca_train_loaded, y_train), 
                         'TEST': (pca_test_loaded, y_test)}],
]

In [24]:
models = []
evaluates = []
i = 0
for name_model in ['rf', 'svm', 'lr', 'knn', 'dt']:
    for experiment in aggregated_data:
        data = experiment[-1]
        classifi_objs = [0] if experiment[2] == 'VALENCE' else [1] if experiment[2] == 'AROUSAL' else [0,1]
        (X_train_data, y_train_data), (X_test_data, y_test_data) = data['TRAIN'], data['TEST']
        # print(X_test_data.shape, X_train_data.shape)
        for classifi_obj in classifi_objs:
            config = {
                'length_signal': X_train_data.shape[-1],
                'name_model': name_model
            }
            
            model = get_model(**config)

            # print(name_model)
            config_train = setup_config_train(X_train_data, y_train_data, X_test_data, y_test_data, name_model)
            train_model(model, classifi_obj, **config_train)

            motion = 'VALENCE' if classifi_obj else 'AROUSAL'

            models.append((*experiment[:2], motion, model))

            info = (
                name_model,
                *experiment[:2], 
                motion,
                evaluate(model, name_model, classifi_obj, X_test_data, y_test_data)['Acc']*100
            )
            i += 1
            print(f'experiment {i}: {info}')
            evaluates.append(info)
            
            

experiment 1: ('rf', 'WT', 'YES', 'AROUSAL', 73.828125)
experiment 2: ('rf', 'WT', 'YES', 'VALENCE', 78.90625)
experiment 3: ('rf', 'WT', 'NO', 'AROUSAL', 74.21875)
experiment 4: ('rf', 'WT', 'NO', 'VALENCE', 77.734375)
experiment 5: ('rf', 'PE', 'NO', 'AROUSAL', 78.125)
experiment 6: ('rf', 'PE', 'NO', 'VALENCE', 78.515625)
experiment 7: ('rf', 'NONE', 'YES', 'AROUSAL', 78.515625)
experiment 8: ('rf', 'NONE', 'YES', 'VALENCE', 78.515625)
experiment 9: ('rf', 'NONE', 'NO', 'AROUSAL', 78.515625)
experiment 10: ('rf', 'NONE', 'NO', 'VALENCE', 78.515625)
experiment 11: ('svm', 'WT', 'YES', 'AROUSAL', 77.734375)
experiment 12: ('svm', 'WT', 'YES', 'VALENCE', 78.515625)
experiment 13: ('svm', 'WT', 'NO', 'AROUSAL', 77.734375)
experiment 14: ('svm', 'WT', 'NO', 'VALENCE', 78.515625)
experiment 15: ('svm', 'PE', 'NO', 'AROUSAL', 78.515625)
experiment 16: ('svm', 'PE', 'NO', 'VALENCE', 78.515625)
experiment 17: ('svm', 'NONE', 'YES', 'AROUSAL', 78.515625)
experiment 18: ('svm', 'NONE', 'YES', 

In [25]:
# Select model type based on input
model_type = "CNN" 

# Feature extraction for train and test sets
if model_type == "CNN":
    features_train = feature_extraction_with_rnn_cnn(X_train, y_train, model_type="CNN", k=128)
    features_test = feature_extraction_with_rnn_cnn(X_test, y_test, model_type="CNN", k=128)
elif model_type == "RNN":
    features_train = feature_extraction_with_rnn_cnn(X_train, y_train, model_type="RNN", k=128)
    features_test = feature_extraction_with_rnn_cnn(X_test, y_test, model_type="RNN", k=128)

# Save the extracted features to a pickle file
output_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
filename = f"pca_data_train_test_{model_type.lower()}.pkl"  # File name will include model type

with open(output_path + filename, 'wb') as file:
    pickle.dump({
        'train': features_train,
        'test': features_test
    }, file)

print(f"Data has been saved to file {output_path + filename}")

Data has been saved to file C:\Users\ferri\Downloads\PoliTO\Tesi\DSs\Emotion-Stress\deap-dataset\pca_data_train_test_cnn.pkl


In [26]:
# Save file name
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
if model_type == "CNN":
    filename = "pca_data_train_test_cnn.pkl"
elif model_type == "RNN":
    filename = "pca_data_train_test_rnn.pkl"

# Read data again
with open(prefix_path + filename, 'rb') as file:
    data = pickle.load(file)
    
# print(data)

# Data retrieval
pca_train_loaded = data['train']
pca_test_loaded = data['test']

print("Size pca_train:", pca_train_loaded.shape)
print("Size pca_test:", pca_test_loaded.shape)

Size pca_train: torch.Size([1024, 64, 5120])
Size pca_test: torch.Size([256, 64, 5120])


In [27]:
from mrmr import mrmr_classif
import pandas as pd

def feature_selector(num_features, X, y, classif_obj): # keep last dimension
    flatted_X = X.reshape(-1, X.shape[-1])

    flatted_X = pd.DataFrame(flatted_X)

    flatted_y_new = y
    if len(y.shape) == 2:
        y_new = np.repeat(y[:, np.newaxis, :], X.shape[1], axis=1)
        flatted_y_new = y_new.reshape(-1, y_new.shape[-1])
        
    flatted_y_new = flatted_y_new[:,classif_obj]

    selected_features = mrmr_classif(X=flatted_X, y=flatted_y_new, K=num_features)

    return selected_features

f = feature_selector(64, cwt_train_loaded, y_train, 0)
feature_selections = {
    'PCA': {
        'Valence': feature_selector(64, pca_train_loaded, y_train, 0), #1024x40x8064
        'Arousal': feature_selector(64, pca_train_loaded, y_train, 1) #1024x40x8064
    },
    'WT': {
        'Valence': feature_selector(64, cwt_train_loaded, y_train, 0), #1024x40x128
        'Arousal': feature_selector(64, cwt_train_loaded, y_train, 1) #1024x40x128
    }
}

# Save file name
prefix_output_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
filename = "feature_selections.pkl"

# Save data
with open(prefix_output_path + filename, 'wb') as file:
    pickle.dump(feature_selections, file)

print(f"Data has been saved to file {filename}")

100%|██████████| 64/64 [00:13<00:00,  4.62it/s]

Data has been saved to file feature_selections.pkl


In [28]:
# Save file name
prefix_path = 'C:\\Users\\ferri\\Downloads\\PoliTO\\Tesi\\DSs\\Emotion-Stress\\deap-dataset\\'
filename = "feature_selections.pkl"

# Read data again
with open(prefix_path + filename, 'rb') as file:
    feature_selection_loaded = pickle.load(file)

print(feature_selection_loaded)

{'PCA': {'Valence': [np.int64(4867), np.int64(4606), np.int64(4892), np.int64(4890), np.int64(4894), np.int64(4893), np.int64(4895), np.int64(4868), np.int64(4896), np.int64(4891), np.int64(4869), np.int64(4903), np.int64(4889), np.int64(4884), np.int64(4756), np.int64(4899), np.int64(4883), np.int64(4898), np.int64(4878), np.int64(4902), np.int64(4871), np.int64(4879), np.int64(4901), np.int64(4870), np.int64(4945), np.int64(4877), np.int64(4897), np.int64(4900), np.int64(4876), np.int64(4748), np.int64(4872), np.int64(4880), np.int64(4881), np.int64(4882), np.int64(4887), np.int64(4994), np.int64(4885), np.int64(4752), np.int64(4904), np.int64(4886), np.int64(4888), np.int64(4607), np.int64(4866), np.int64(4875), np.int64(4749), np.int64(4947), np.int64(4873), np.int64(4757), np.int64(4874), np.int64(4944), np.int64(4758), np.int64(4995), np.int64(4822), np.int64(4913), np.int64(4823), np.int64(4948), np.int64(4768), np.int64(4911), np.int64(4750), np.int64(4912), np.int64(4781), np.

In [29]:
aggregated_data = [
    ['WT', 'YES', 'VALENCE', {'TRAIN': (cwt_train_loaded[:, :, feature_selection_loaded['WT']['Valence']], y_train), 
                              'TEST': (cwt_test_loaded[:, :, feature_selection_loaded['WT']['Valence']], y_test)}],
    ['WT', 'YES', 'AROUSAL', {'TRAIN': (cwt_train_loaded[:, :, feature_selection_loaded['WT']['Arousal']], y_train),
                              'TEST': (cwt_test_loaded[:, :, feature_selection_loaded['WT']['Arousal']], y_test)}],

    ['WT', 'NO', '*', {'TRAIN': (cwt_train_loaded, y_train), 
                       'TEST': (cwt_test_loaded, y_test)}],

    ['PE', 'NO', '*', {'TRAIN': (pe_train_loaded, y_train), 
                       'TEST': (pe_test_loaded, y_test)}],

    ['NONE', 'YES', 'VALENCE', {'TRAIN': (pca_train_loaded[:, :, feature_selection_loaded['PCA']['Valence']], y_train), 
                                'TEST': (pca_test_loaded[:, :, feature_selection_loaded['PCA']['Valence']], y_test)}],
    ['NONE', 'YES', 'AROUSAL', {'TRAIN': (pca_train_loaded[:, :, feature_selection_loaded['PCA']['Arousal']], y_train), 
                                'TEST': (pca_test_loaded[:, :, feature_selection_loaded['PCA']['Arousal']], y_test)}],

    ['NONE', 'NO', '*', {'TRAIN': (pca_train_loaded, y_train), 
                         'TEST': (pca_test_loaded, y_test)}],
]

In [30]:
models = []
evaluates = []
i = 0
for name_model in ['rf', 'svm', 'lr', 'knn', 'dt']:
    for experiment in aggregated_data:
        data = experiment[-1]
        classifi_objs = [0] if experiment[2] == 'VALENCE' else [1] if experiment[2] == 'AROUSAL' else [0,1]
        (X_train_data, y_train_data), (X_test_data, y_test_data) = data['TRAIN'], data['TEST']
        # print(X_test_data.shape, X_train_data.shape)
        for classifi_obj in classifi_objs:
            config = {
                'length_signal': X_train_data.shape[-1],
                'name_model': name_model
            }
            
            model = get_model(**config)

            # print(name_model)
            config_train = setup_config_train(X_train_data, y_train_data, X_test_data, y_test_data, name_model)
            train_model(model, classifi_obj, **config_train)

            motion = 'VALENCE' if classifi_obj else 'AROUSAL'

            models.append((*experiment[:2], motion, model))

            info = (
                name_model,
                *experiment[:2], 
                motion,
                evaluate(model, name_model, classifi_obj, X_test_data, y_test_data)['Acc']*100
            )
            i += 1
            print(f'experiment {i}: {info}')
            evaluates.append(info)
            
            

experiment 1: ('rf', 'WT', 'YES', 'AROUSAL', 73.828125)
experiment 2: ('rf', 'WT', 'YES', 'VALENCE', 78.90625)
experiment 3: ('rf', 'WT', 'NO', 'AROUSAL', 74.21875)
experiment 4: ('rf', 'WT', 'NO', 'VALENCE', 77.734375)
experiment 5: ('rf', 'PE', 'NO', 'AROUSAL', 78.125)
experiment 6: ('rf', 'PE', 'NO', 'VALENCE', 78.515625)
experiment 7: ('rf', 'NONE', 'YES', 'AROUSAL', 69.140625)
experiment 8: ('rf', 'NONE', 'YES', 'VALENCE', 78.90625)
experiment 9: ('rf', 'NONE', 'NO', 'AROUSAL', 78.515625)
experiment 10: ('rf', 'NONE', 'NO', 'VALENCE', 22.65625)
experiment 11: ('svm', 'WT', 'YES', 'AROUSAL', 77.734375)
experiment 12: ('svm', 'WT', 'YES', 'VALENCE', 78.515625)
experiment 13: ('svm', 'WT', 'NO', 'AROUSAL', 77.734375)
experiment 14: ('svm', 'WT', 'NO', 'VALENCE', 78.515625)
experiment 15: ('svm', 'PE', 'NO', 'AROUSAL', 78.515625)
experiment 16: ('svm', 'PE', 'NO', 'VALENCE', 78.515625)
experiment 17: ('svm', 'NONE', 'YES', 'AROUSAL', 78.515625)
experiment 18: ('svm', 'NONE', 'YES', 'V

# **Summary of Best Scores and Configurations**

## **Best Scores for Each Model:**

| Model      | Classifier | Feature Set | Data Augmentation | Task   | Accuracy (%) |
|------------|-----------|-------------|-------------------|--------|-------------|
| **RNN LSTM** | SVM       | NONE        | YES               | AROUSAL | **78.5156** |
| **RNN LSTM** | SVM       | NONE        | YES               | VALENCE | **78.5156** |
| **RNN GRU**  | SVM       | NONE        | YES               | AROUSAL | **78.5156** |
| **RNN GRU**  | SVM       | NONE        | YES               | VALENCE | **78.5156** |
| **CNN**      | RF        | WT          | YES               | VALENCE | **78.9063** |

---

## **Best Configurations per Model Type:**

### **RNN LSTM:**
- **Best Score:** **78.5156%**
- **Best Configuration:**  
  - Classifier: **SVM**
  - Feature Set: **NONE**
  - Data Augmentation: **YES**
  - Works best for both **Arousal and Valence**

### **RNN GRU:**
- **Best Score:** **78.5156%**
- **Best Configuration:**  
  - Classifier: **SVM**
  - Feature Set: **NONE**
  - Data Augmentation: **YES**
  - Works best for both **Arousal and Valence**

### **CNN:**
- **Best Score:** **78.9063%** (Highest overall)
- **Best Configuration:**  
  - Classifier: **Random Forest (RF)**
  - Feature Set: **WT**
  - Data Augmentation: **YES**
  - Works best for **Valence**

---

## **Key Takeaways:**
- **Best overall performance is achieved by CNN with RF classifier (78.9063% for Valence).**
- **SVM performs best for RNN models, especially with no predefined feature set (NONE) and data augmentation (YES).**
- **Data augmentation ("YES") appears to contribute to higher accuracy.**
- **Feature Set "NONE" (no predefined features) seems to work well for RNN models.**
- **Random Forest (RF) is the top performer for CNN.**
